In [1]:
import os
import numpy as np
import pandas as pd
from scipy.interpolate import interp1d
import sys
import math
from netCDF4 import Dataset
import os, calendar, sys, fnmatch, datetime
from sur_spec_alb_interpolate import sur_spec_alb_interpolate

from INPUT_RRTM_sw import *
from INPUT_CLD_RRTM_sw import *
from INPUT_AER_RRTM_sw import *

In [2]:
#*******************
model_dir = '/umbc/xfs1/zzbatmos-new/common/Codes/'
output_dir = './rrtm_output/'
RT_model_sw = model_dir+'rrtm_sw/rrtm_sw_taki_intel'
RT_model_lw = model_dir+'rrtm_lw/rrtm_lw_taki_intel'

#Parameters the model needs
SOLVAR  = 0.0  #(16,29)the solar source function scale factor for each band.
#HBOUND = np.loadtxt(fname=file6)       #altitude of the surface (km)
HTOA   = 77.0       #altitude of the top of the atmosphere (km)

#--------------------------------------------------;
ISCAT_lw   = 2
ISCAT_sw   = 0
NUMANGS = 0 # 4 streams for lw
ISTRM = 0   # 4 streams for sw
NSTR  = 4   # of streams

In [3]:
# =====RRTM input data=====
atm_prof = np.loadtxt('atm_profile_cloudysky.txt',skiprows=1)
ZM = atm_prof[::-1,0] #km
PM = atm_prof[::-1,1] #hpa
TM = atm_prof[::-1,2] #K
h2o = atm_prof[::-1,4]  #g/m3   #kg/kg . kg/kg *1e3 = gm/kg
o3 = atm_prof[::-1,5]   #g/m3  #ppmv 'parts per million volume = 1e-6'. 1ppmv=1e-3*gm/kg

Rs = np.array([0.1]) ###
sza = np.array([0]) ###

In [4]:
n_case = np.size(Rs)
print(n_case)
for ic in range(n_case):
    SUF_ALB = np.ones(14)*Rs[ic]###

    print(ZM)
    print(SUF_ALB)
    
    SZA = sza[ic]
    if ZM[0]<0.1 :
        HBOUND = ZM[0]+0.001
    else:
        HBOUND = ZM[0]+0.01
    IMMAX = len(PM)
    IBMAX = IMMAX

    VMOL_wv = h2o   ###kg/kg*1e3=gm/kg
    VMOL_o3 = o3    ###ppmv*1e3=gm/kg
    VMOL_co2= np.ones(IMMAX)* 360 #no CO2 profile in data, add mannually, ppmv
    VMOL_n2o = np.ones(IMMAX)*0.3 #PPMV
    VMOL_co = np.zeros(IMMAX)
    VMOL_ch4 = np.ones(IMMAX)*1.7 #PPMV
    VMOL_total = np.vstack((VMOL_wv,VMOL_co2,VMOL_o3,VMOL_n2o,VMOL_co,VMOL_ch4))#density of the molecule set by JCHAR(K)
    VMOL=VMOL_total

    #---------------sw flux-------------------------------------
    ICLD = 0
    IAER = 0
    print(' Compute clear-sky sw flux w/o aerososl')
    read_input_rrtm_sw('INPUT_RRTM', IAER, ICLD, ISCAT_sw, ISTRM, SZA, 1.0-SUF_ALB, \
    HBOUND, IMMAX, ZM, PM, TM, VMOL)
    os.system(RT_model_sw)
    os.system('mv INPUT_RRTM INPUT_RRTM.sw')
    os.system('mv OUTPUT_RRTM '+output_dir+'OUTPUT_RRTM_sw.clear')

    #!!layer n corresponds to the region between altitudes n and n+1 in the list of layer boundaries
    CWP=atm_prof[::-1,6] #cloud water path (g/m^2)
    EFFSIZELIQ=atm_prof[::-1,7] #cloud droplet effective radius (microns)
    NLAY=np.count_nonzero(CWP) # A scalar represents the number of cloudy layers
    LAY=np.where(CWP!=0)[0] # An array contains index of cloudy layers
    print(CWP)
    print(EFFSIZELIQ)
    print('***',len(CWP),NLAY, LAY)

    ICLD = 1
    IAER = 0
    print(' Compute sw fluxs with clouds')
    read_input_rrtm_sw('INPUT_RRTM', IAER, ICLD, ISCAT_sw, ISTRM, SZA, 1.0-SUF_ALB, \
    HBOUND, IMMAX, ZM, PM, TM, VMOL)
    write_IN_CLD_RRTM_sw('IN_CLD_RRTM', NLAY, LAY, CWP, EFFSIZELIQ)
    os.system(RT_model_sw)
    os.system('mv INPUT_RRTM INPUT_RRTM.sw')
    os.system('mv IN_CLD_RRTM IN_CLD_RRTM.sw')
    os.system('mv OUTPUT_RRTM '+output_dir+'OUTPUT_RRTM_sw.clouds')

1
[ 0.  1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12. 13. 14. 15. 16. 17.
 18. 19. 20. 21. 22. 23. 24. 25. 30. 35. 40. 45. 50.]
[0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1]
 Compute clear-sky sw flux w/o aerososl
[ 0.  16.7 16.7  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
  0.   0.   0. ]
[ 0. 10. 10.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
*** 31 2 [1 2]
 Compute sw fluxs with clouds
*** 1 16.7 10.0
*** 2 16.7 10.0
